# DATA 512 - Project Part 1: Common Analysis

## Wildfires Analysis - Data Retrieval

### Tanushree Yandra, University of Washington, Seattle

More and more frequently, summers in the western US have been characterized by wildfires with smoke billowing across multiple western states. There are many proposed causes for this: climate change, US Forestry policy, growing awareness, just to name a few. Regardless of the cause, the impact of wildland fires is widespread. There is a growing body of work pointing to the negative impacts of smoke on health, tourism, property, and other aspects of society. This project analyzes wildfire impacts on the city of Twin Falls, Idaho in the US. The end goal is to be able to inform policy makers, city managers, city councils, or other civic institutions, to make an informed plan for how they could or whether they should make plans to mitigate future impacts from wildfires.

Wildland fires within 1250 miles of Twin Falls, Idaho are analyzed for the last 60 years (1963-2020). A smoke estimate is then created to estimate the wildfire smoke impact which is later modeled to make predictions for the next 30 years (until 2049).

This section of the notebook discusses the data retrieval process. Two datasets are needed for the analysis - Wildland Fires Dataset to get access to the historical data of wildfires, and the Air Quality Index Dataset to validate our smoke estimate. The data acquisition process for both of these datasets has been explained in detail below.

## 1. Retrieving the Wildland Fires Dataset

The [Combined Wildland Fire Datasets for the United States and certain territories, 1800s-Present (combined wildland fire polygons)](https://www.sciencebase.gov/catalog/item/61aa537dd34eb622f699df81) dataset was used to retrieve the historical data of wildfires. This dataset was collected and aggregated by the US Geological Survey, and is relatively well documented. Fire polygons in this dataset are available in ArcGIS and GeoJSON formats.

For this analysis, the files were downloaded in the GeoJSON format and a GeoJSON reader was used to read the data. The GeoJSON reader has been sampled from this [module](https://drive.google.com/file/d/1TwCkvdaw0MxJzW7NSDg6XxYQ0dvaS44I/view?usp=sharing). The 'wildfire' module is a user module. This module is available from the course website. The module includes one object, a Reader, that can be used to read the GeoJSON files associated with the wildefire dataset. The module also contains a sample datafile that is GeoJSON compliant and that contains a small number of California wildfires extracted from the main wildfire dataset. This sample code is provided under copyright by the author and cannot be redistributed. For any permissions, please reach out to the author -  Dr. David W. McDonald.

While reading the data, geodetic distance computations were simultaneously being made to filter those wildfires that are within 1250 miles of Twin Falls, Idaho. The functions to make these calculations have been sampled from this [sample notebook](https://drive.google.com/file/d/1qNI6hji8CvDeBsnLDAhJXvaqf2gcg8UV/view?usp=sharing). This notebook is licensed under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/).

### Step 1: Preliminaries

First, we start by importing required modules and packages. For the wildfire module, it is assumed that you have set an appropriate Python path environment variable to point to the location on your machine where you store python user modules.

In [1]:
# These are standard python modules
import pandas as pd
import json
import warnings

# The modules below are not standard Python modules
# You will need to install these with pip/pip3 if you do not already have it
from pyproj import Transformer, Geod

# The module below is the GeoJSON reader that has been sampled
from wildfire import Reader

In [2]:
# Suppress the warning statements
warnings.filterwarnings("ignore")

### Step 2: Reading the GeoJSON Data

First, load the downloaded GeoJSON data into a JSON object.

In [3]:
# Open the GeoJSON file in a JSON object
with open('USGS_Wildland_Fire_Combined_Dataset.json', 'r') as json_file:
    wildfire_data = json.load(json_file)

We then create a wildfire Reader() object and use it to open the wildfires dataset.

In [4]:
# This opens the named file, reading the file header information
# This sets up the file to start reading features uing the next() method
reader = Reader.Reader()
reader.open("USGS_Wildland_Fire_Combined_Dataset.json")

Once opened, we have access to the header information so we print that to show the structure of that data.

In [5]:
# This outputs the header information that was read from the GeoJSON file when it was opened
reader.header()

{'displayFieldName': '',
 'fieldAliases': {'OBJECTID': 'OBJECTID',
  'USGS_Assigned_ID': 'USGS Assigned ID',
  'Assigned_Fire_Type': 'Assigned Fire Type',
  'Fire_Year': 'Fire Year',
  'Fire_Polygon_Tier': 'Fire Polygon Tier',
  'Fire_Attribute_Tiers': 'Fire Attribute Tiers',
  'GIS_Acres': 'GIS_Acres',
  'GIS_Hectares': 'GIS_Hectares',
  'Source_Datasets': 'Source Datasets',
  'Listed_Fire_Types': 'Listed Fire Types',
  'Listed_Fire_Names': 'Listed Fire Names',
  'Listed_Fire_Codes': 'Listed Fire Codes',
  'Listed_Fire_IDs': 'Listed Fire IDs',
  'Listed_Fire_IRWIN_IDs': 'Listed Fire IRWIN IDs',
  'Listed_Fire_Dates': 'Listed Fire Dates',
  'Listed_Fire_Causes': 'Listed Fire Causes',
  'Listed_Fire_Cause_Class': 'Listed Fire Cause Class',
  'Listed_Rx_Reported_Acres': 'Listed Rx Reported Acres',
  'Listed_Map_Digitize_Methods': 'Listed Map Digitize Methods',
  'Listed_Notes': 'Listed Notes',
  'Processing_Notes': 'Processing Notes',
  'Wildfire_Notice': 'Wildfire Notice',
  'Prescribed

### Step 3: Converting Points between Geodetic Coordinate Systems

Now that we are able to read the GeoJSON data, we will now have to define functions to compute the geodetic distance between a wildfire and the city of Twin Falls, Idaho. One of the constraints in doing geodetic computations is that most of the time we need to have our points (the coordinates for places) in the same geographic coordinate system. There are tons and tons of coordinate systems. You can find descriptions of many of them at [EPSG.io](https://epsg.io).

Looking at the wildfire header information, we can see fields named "geometryType" and "spatialReference". This looks like:

        "geometryType": "esriGeometryPolygon",
        "spatialReference": {
            "wkid": 102008,
            "latestWkid": 102008
        },

This indicates that the geometry of our wildfire data are generic polygons and that they are expressed in a coordinate system with the well-known ID (WKID) 102008. This coordinate system is also known as [ESRI:102008](https://epsg.io/102008)

The most commonly used geographic coordinate system however is 'WGS84', that is sometimes called 'decimal degrees' (DD). This decimal degrees system has an official name (or WKID) of [EPSG:4326](https://epsg.io/4326). The coordinates of our city - Twin Falls, Idaho are stored in this format.

Thus, in order to compute the distance, we will first need to convert the coordinates to the Decimal Degrees system. We are going to do this by taking the geometry of a fire feature, extracting the largest ring (i.e., the largest boundary of the fire) and converting all of the points in that ring from the ESRI:102008 coordinate system to EPSG:4326 coordinates.

In [6]:
# FUNCTION

# The function takes one parameter, a list of ESRI:102008 coordinates that will be transformed to EPSG:4326
# The function returns a list of coordinates in EPSG:4326

def convert_ring_to_epsg4326(ring_data=None):
    
    # Initiate an empty list
    converted_ring = list()

    # Pyproj transformer is used to convert coordinates from ESRI:102008 to EPSG:4326
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    
    # Run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0],coord[1])
        new_coord = lat,lon
        # Append the converted coordinates to the list
        converted_ring.append(new_coord)
    return converted_ring

### Step 4: Computing the Geodetic Distance

The basic problem is knowing how far away a fire is from some particular location. One issue is that fires are irregularly shaped so the actual answer to that is a bit dependent upon the exact shape and how you want to think about the notion of 'distance'. For example, should we just find the closest point on the perimiter of a fire and call that the distance? Maybe we should find the centroid of the region, identify that as a geolocation (coordinate) and then calculate the distance to that? We can come up with numerous other ways.

For this analysis, the average distance of all perimeter points to Twin Falls, Idaho was considered. This is not quite what the centroid would be, but it is probably fairly close. This method was chosen because for some very large fires, including just the shortest distance seemed a little biased. Average distance felt like a more rounded and appropriate approach. The function below shows how this distance can be computed.

In [7]:
# FUNCTION

# The function takes two parameters
# place - coordinates as a list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
# ring_data - a list of decimal degree coordinates for the fire boundary

# The function returns the average miles from boundary to the place

def average_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    
    # Convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)
    
    # Create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    
    # Create a list to store our results
    distances_in_meters = list()
    
    # Run through each point in the converted ring data
    for point in ring:
        
        # Calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        distances_in_meters.append(d[2])

    # Convert meters to miles
    distances_in_miles = [meters*0.00062137 for meters in distances_in_meters]
    
    # The ring requires that the first and last coordinates be identical to close the polygon region
    # We thus remove one of them so that we don't bias our average by having two of the same point
    distances_in_miles_no_dup = distances_in_miles[1:]
    
    # Now, compute the average miles
    average = sum(distances_in_miles_no_dup)/len(distances_in_miles_no_dup)
    return average

### Step 5: Generating the Final Wildfires Dataset

In this section, we use the Reader() object and the next() method to read all the wildfire features in the dataset. For this analysis, every 'feature' is a wildfire. While we read individual 'features' we also access the geometry of each feature to get the 'ring' boundary of that specific fire - which is a list of geodetic coordinates. These coordinates allow us to compute the distance between the wildfire and our city of interest.

Some of the wildfire features contain 'curveRings' in the feature geometry instead of 'rings'. Such wildfires are very less (< ~50) and thus have been ignored for the purpose of this analysis.

In [8]:
# Define the coordinates for your city of interest (Twin Falls, Idaho in our case)
city_latlon = [42.562786, -114.460503]

# Initiate an empty list to store the wildfires data
final_wildfire_data = []
# Set the pointer to the first feature of the dataset
wildfire_feature = reader.next()

# Iterate through every feature until it return an empty value which indicates the file has ended
while wildfire_feature is not None:
    
    # Extract the fire year from the attributes of the feature
    fire_year = wildfire_feature['attributes']['Fire_Year']
    
    # Ignore wildfires before the year 1963
    if fire_year >= 1963:
        
        # Get the geometry for the feature we pulled from the feature_list
        wildfire_geometry = wildfire_feature['geometry']
        
        # Ignore features that have 'curveRings' instead of 'rings'
        if list(wildfire_feature['geometry'].keys())[0] == 'curveRings':
            wildfire_feature = reader.next()
            continue;

        # Get all the coordinates for the fire boundary
        ring_data = wildfire_feature['geometry']['rings'][0]
        
        # Compute the average distance between the fire boundary and Twin Falls, Idaho
        distance = average_distance_from_place_to_fire_perimeter(city_latlon,ring_data)
        
        # Only consider those wildfires that are within 1250 miles of Twin Falls, Idaho
        if distance <= 1250:
            
            # Save the distance value in the feature attributes and append the feature to the list
            wildfire_feature['attributes']['Distance'] = distance
            final_wildfire_data.append(wildfire_feature)
    
    # Move to the next feature
    wildfire_feature = reader.next()

In [9]:
# Look at the total features in the dataset
len(final_wildfire_data)

84319


Thus, we have a total of 84319 wildfires in the period 1963-2020 where all the wildfires are within 1250 miles of Twin Falls, Idaho.

In [10]:
# Save the list of dictionaries in a json file
with open('final_wildfire_data.json', 'w') as json_file:
    json.dump(final_wildfire_data, json_file, indent=4)

## 2. Retrieving US EPA Air Quality Data

The code below illustrates how to request data from the US Environmental Protection Agency (EPA) Air Quality Service (AQS) API. This is a historical API and does not provide real-time air quality data. The [documentation](https://aqs.epa.gov/aqsweb/documents/data_api.html) for the API provides definitions of the different call parameter and examples of the various calls that can be made to the API.

The code for the data retrieval has been sampled from this [sample notebook](https://drive.google.com/file/d/1bxl9qrb_52RocKNGfbZ5znHVqFDMkUzf/view?usp=sharing) which is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). The sampled code works systematically through requesting an API key, using 'list' to get various IDs and parameter values, and using 'daily summary' to get summary data that meets specific condistions. Changing values to explore the results of the API is probably useful, but that will result in some explanations being out of sync with the outputs.

The US EPA was created in the early 1970's. The EPA reports that they only started broad based monitoring with standardized quality assurance procedures in the 1980's. Many counties will have data starting somewhere between 1983 and 1988. However, some counties still do not have any air quality monitoring stations. The API helps resolve this by providing calls to search for monitoring stations and data using either station ids, or a county designation or a geographic bounding box. The code below shows how the Air Quality Index or AQI data can be retrieved using a county designation. Some [additional information on the Air Quality System can be found in the EPA FAQ](https://www.epa.gov/outdoor-air-quality-data/frequent-questions-about-airdata) on the system.

The end goal of this code is to get to the AQI data for the city of Twin Falls, Idaho. The AQI index is meant to tell us something about how healthy or clean the air is on any day. The AQI is actually a somewhat complex measure. To understand what parameters go into calculating the AQI, check [how to calculate the AQI](https://www.airnow.gov/sites/default/files/2020-05/aqi-technical-assistance-document-sept2018.pdf).

### Step 1: Preliminaries

First, we start by importing required modules and packages, and defining some constants that will be used later for the API call.

In [11]:
# These are standard python modules
import json, time
import numpy as np

# The modules below are not standard Python modules
# You will need to install these with pip/pip3 if you do not already have it
import requests

In [12]:
# CONSTANTS

API_REQUEST_URL = 'https://aqs.epa.gov/data/api'

# These are 'actions' we can ask the API to take or requests that we can make of the API
# Sign-up request - generally only performed once - unless you lose your key
API_ACTION_SIGNUP = '/signup?email={email}'

# List actions provide information on API parameter values that are required by other actions/requests
API_ACTION_LIST_CLASSES = '/list/classes?email={email}&key={key}'
API_ACTION_LIST_PARAMS = '/list/parametersByClass?email={email}&key={key}&pc={pclass}'
API_ACTION_LIST_SITES = '/list/sitesByCounty?email={email}&key={key}&state={state}&county={county}'

# Monitor actions are requests for monitoring stations that meet specific criteria
API_ACTION_MONITORS_COUNTY = '''/monitors/byCounty?email={email}&key={key}&param={param}&
bdate={begin_date}&edate={end_date}&state={state}&county={county}'''
API_ACTION_MONITORS_BOX = '''/monitors/byBox?email={email}&key={key}&param={param}&bdate={begin_date}
&edate={end_date}&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}'''

# Summary actions are requests for summary data. These are for daily summaries
API_ACTION_DAILY_SUMMARY_COUNTY = '''/dailyData/byCounty?email={email}&key={key}&param={param}&
bdate={begin_date}&edate={end_date}&state={state}&county={county}'''
API_ACTION_DAILY_SUMMARY_BOX = '''/dailyData/byBox?email={email}&key={key}&param={param}&
bdate={begin_date}&edate={end_date}&minlat={minlat}&maxlat={maxlat}&minlon={minlon}&maxlon={maxlon}'''

# It is always nice to be respectful of a free data resource
# We are going to observe a 100 requests per minute limit - which is fairly nice
# Assuming roughly a 2ms latency on the API and network
API_LATENCY_ASSUMED = 0.002       
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# This is a template that covers most of the parameters for the actions we might take
AQS_REQUEST_TEMPLATE = {
    "email":      "",     
    "key":        "",      
    "state":      "",     # the two digit state FIPS # as a string
    "county":     "",     # the three digit county FIPS # as a string
    "begin_date": "",     # the start of a time window in YYYYMMDD format
    "end_date":   "",     # the end of a time window in YYYYMMDD format (both dates must have same year)
    "minlat":    0.0,
    "maxlat":    0.0,
    "minlon":    0.0,
    "maxlon":    0.0,
    "param":     "",      # a list of comma separated 5 digit codes, max 5 codes requested
    "pclass":    ""       # parameter class is only used by the List calls
}

### Step 2: Making a Sign-Up Request

Before you can use the API you need to request a key. You will use an email address to make the request. The EPA then sends a confirmation email link and a 'key' that you use for all other requests.

You only need to sign-up once, unless you want to invalidate your current key (by getting a new key) or you lose your key.

In [13]:
# FUNCTION

# This implements the sign-up request
# The parameters are standardized so that this function definition matches all of the others
# The easiest way to call this is to simply call this function with your preferred email address.

def request_signup(email_address = None,
                   endpoint_url = API_REQUEST_URL, 
                   endpoint_action = API_ACTION_SIGNUP, 
                   request_template = AQS_REQUEST_TEMPLATE,
                   headers = None):
    
    # If you don't have access to this email addres, things might go badly for you
    if email_address:
        request_template['email'] = email_address        
    if not request_template['email']: 
        raise Exception("Must supply an email address to call 'request_signup()'")
    
    # Compose the signup url
    # Create a request URL by combining the endpoint_url with the parameters for the request
    request_url = endpoint_url+endpoint_action.format(**request_template)
        
    # Make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in case an exception occurs
        # During the request processing, throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

In [14]:
# A SIGNUP request is only to be done once, to request a key
# A key is sent to that email address and needs to be confirmed with a click
# This code should be commented out after you've made your key request
# This will make sure you don't accidentally make a new sign-up request

#response = request_signup("tyandra@uw.edu")

Assign your email address to "USERNAME" and your key to "APIKEY" as constants.

In [15]:
# Once we have the signup email, we can define two constants
# USERNAME - the email address you sent the EPA asking for access to the API during sign-up
# APIKEY - This should be the authorization key they sent you

# You can specify these as constants for your own use
# Just don't distribute the notebook without removing the constants

#USERNAME = "<the_email_address_you_sent_on_signup>"
#APIKEY = "<the_key_the_EPA_sent_you_in_email>"

### Step 3: Making a List Request

Once you have a key, the next thing is to get information about the different types of air quality monitoring (sensors) and the different places where we might find air quality stations. The monitoring system is complex and changes all the time. The EPA implementation allows an API user to find changes to monitoring sites and sensors by making requests - maybe monthly, or daily. This API approach is probably better than having the EPA publish documentation that may be out of date as soon as it hits a web page. The one problem here is that some of the responses rely on jargon or terms-of-art. That is, one needs to know a bit about the way atmospheric sciece works to understand some of the terms. Good thing we can use the web to search for terms we don't know!

In [16]:
# FUNCTION

# This implements the list request
# There are several versions of the list request that only require email and key
# This code sets the default action/requests to list the groups or parameter class descriptors
# This enables to request the 5 digit codes for individual air quality measures using the param request

def request_list_info(email_address = None, key = None,
                      endpoint_url = API_REQUEST_URL, 
                      endpoint_action = API_ACTION_LIST_CLASSES, 
                      request_template = AQS_REQUEST_TEMPLATE,
                      headers = None):
    
    #  Make sure we have email and key - at least
    #  This prioritizes the info from the call parameters - not what's already in the template
    if email_address:
        request_template['email'] = email_address
    if key:
        request_template['key'] = key
    
    # For the basic request we need an email address and a key
    if not request_template['email']:
        raise Exception("Must supply an email address to call 'request_list_info()'")
    if not request_template['key']: 
        raise Exception("Must supply a key to call 'request_list_info()'")

    # Compose the request
    request_url = endpoint_url+endpoint_action.format(**request_template)
        
    # Make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in the situation where an exception occurs
        # during the request processing - throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

In [17]:
# The default should get us a list of the various groups or classes of sensors
# These classes are user defined names for clustors of sensors
# We need a class name to start getting down to the sensor ID
# Each sensor type has an ID number
# We'll eventually need those ID numbers to be able to request values from that specific sensor

request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY

# Make a list request
response = request_list_info(request_template=request_data)

if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))

[
    {
        "code": "AIRNOW MAPS",
        "value_represented": "The parameters represented on AirNow maps (88101, 88502, and 44201)"
    },
    {
        "code": "ALL",
        "value_represented": "Select all Parameters Available"
    },
    {
        "code": "AQI POLLUTANTS",
        "value_represented": "Pollutants that have an AQI Defined"
    },
    {
        "code": "CORE_HAPS",
        "value_represented": "Urban Air Toxic Pollutants"
    },
    {
        "code": "CRITERIA",
        "value_represented": "Criteria Pollutants"
    },
    {
        "code": "CSN DART",
        "value_represented": "List of CSN speciation parameters to populate the STI DART tool"
    },
    {
        "code": "FORECAST",
        "value_represented": "Parameters routinely extracted by AirNow (STI)"
    },
    {
        "code": "HAPS",
        "value_represented": "Hazardous Air Pollutants"
    },
    {
        "code": "IMPROVE CARBON",
        "value_represented": "IMPROVE Carbon Parameters"
    }

We are interested in getting to something that might be the Air Quality Index (AQI). You see this reported on the news - often around smog values, but also when there is smoke in the sky. The AQI is a complex measure of different gases and particles in the air (dust, dirt, ash, etc.).

From the list produced by our 'list/Classes' request above, it looks like there is a class of sensors called "AQI POLLUTANTS". Let's try to get a list of those specific sensors and see what we can get from those.

In [18]:
# We can find the sensor IDs that make up the class (group) we identified above
# The one that looks to be associated with the Air Quality Index is "AQI POLLUTANTS"
# We'll use that to make another list request

AQI_PARAM_CLASS = "AQI POLLUTANTS"

In [19]:
# Structure a request to get the sensor IDs associated with the AQI

request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY

# Here we specify that we want this 'pclass' or parameter classs
request_data['pclass'] = AQI_PARAM_CLASS

# Make a list request
response = request_list_info(request_template=request_data, endpoint_action=API_ACTION_LIST_PARAMS)

if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))

[
    {
        "code": "42101",
        "value_represented": "Carbon monoxide"
    },
    {
        "code": "42401",
        "value_represented": "Sulfur dioxide"
    },
    {
        "code": "42602",
        "value_represented": "Nitrogen dioxide (NO2)"
    },
    {
        "code": "44201",
        "value_represented": "Ozone"
    },
    {
        "code": "81102",
        "value_represented": "PM10 Total 0-10um STP"
    },
    {
        "code": "88101",
        "value_represented": "PM2.5 - Local Conditions"
    },
    {
        "code": "88502",
        "value_represented": "Acceptable PM2.5 AQI & Speciation Mass"
    }
]


We should now have (above) a response containing a set of sensor ID numbers. The list should include the sensor numbers as well as a description or name for each sensor.

The EPA AQS API has limits on some call parameters. Specifically, when we request data for sensors we can only specify a maximum of 5 different sensor values to return. This means we cannot get all of the Air Quality Index parameters in one request for data. We have to break it up. This can be achieved by breaking the request into two logical groups, the AQI sensors that sample gasses and the AQI sensors that sample particles in the air.

In [20]:
# Given the sensor codes, we can create a parameter list as defined by the AQS API
# It turns out that we want all of these measures for AQI
# But we need to have two different param constants to get all seven of the code types
# We can only have a max of 5 sensors/values request per param
# Thus, they are divided into two group - gaseous and particulate

# Gaseous AQI pollutants CO, SO2, NO2, and O2
AQI_PARAMS_GASEOUS = "42101,42401,42602,44201"

# Particulate AQI pollutants PM10, PM2.5, and Acceptable PM2.5
AQI_PARAMS_PARTICULATES = "81102,88101,88502"

Air quality monitoring stations are located all over the US at different locations. We will need some sample locations to experiment with different locations to see what kinds of values come back from different sensor requests.

This list includes the [FIPS](https://www.census.gov/library/reference/code-lists/ansi.html) number for the state and county as a 5 digit string. This format, the 5 digit string, is a 'old' format that is still widely used. There are new codes that may eventually be adopted for the US government information systems. But FIPS is currently what the AQS uses, so that's what is in the list as the constant.

In [21]:
# Save the FIPS code and latitude, longitude details of Twin Falls, Idaho in a dictionary

CITY = {'city' : 'Twin Falls',
        'county' : 'Twin Falls',
        'state'  : 'Idaho',
        'fips'   : '16083',
        'latlon' : [42.562786, -114.460503]}

Given our city constants, we can now find which monitoring locations are nearby. This code uses the county to define the area we are interested in. You can get the EPA to list their monitoring stations by county.

In [22]:
# This list request gives a list of all monitoring stations in the county specified by CITY dictionary

request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['state'] = CITY['fips'][:2]   # the first two digits (characters) of FIPS is the state code
request_data['county'] = CITY['fips'][2:]  # the last three digits (characters) of FIPS is the county code

# Make a list request
response = request_list_info(request_template=request_data, endpoint_action=API_ACTION_LIST_SITES)

if response["Header"][0]['status'] == "Success":
    print(json.dumps(response['Data'],indent=4))
else:
    print(json.dumps(response,indent=4))

[
    {
        "code": "0001",
        "value_represented": null
    },
    {
        "code": "0002",
        "value_represented": null
    },
    {
        "code": "0003",
        "value_represented": null
    },
    {
        "code": "0004",
        "value_represented": null
    },
    {
        "code": "0005",
        "value_represented": "ROOF OF LARGE RETAIL STORE IN CENTRAL TWIN FALLS"
    },
    {
        "code": "0006",
        "value_represented": "BACKGROUND SITE IN REMOTE HIGH DESERT AREA WEST OF ROGERSON, ID"
    },
    {
        "code": "0007",
        "value_represented": "Twin Falls"
    },
    {
        "code": "0009",
        "value_represented": null
    },
    {
        "code": "0010",
        "value_represented": "MONITOR IS ON ROOF OF SMITH'S FOOD STORE, IN RESIDENTIAL SECTION."
    },
    {
        "code": "1001",
        "value_represented": null
    }
]


The above response gives us a list of monitoring stations. Each monitoring station has a unique "code" which is a string number, and, sometimes, a description. The description seems to be something about where the monitoring station is located.

### Step 4: Making a Daily Summary Request

The function below is designed to encapsulate requests to the EPA AQS API. When calling the function, one should create/copy a parameter template, then initialize that template with values that won't change with each call. Then on each call, simply pass in the parameters that need to change, like date ranges.

In [23]:
# FUNCTION

# This implements the daily summary request
# Daily summary provides a summary value for each sensor from the start date to the end date
# This can be called with a mixture of a defined parameter dictionary, or with function parameters
# If function parameters are provided, they take precedence over parameters from the request template

def request_daily_summary(email_address = None, key = None, param=None,
                          begin_date = None, end_date = None, fips = None,
                          endpoint_url = API_REQUEST_URL, 
                          endpoint_action = API_ACTION_DAILY_SUMMARY_COUNTY, 
                          request_template = AQS_REQUEST_TEMPLATE,
                          headers = None):
    
    #  This prioritizes the info from the call parameters - not what's already in the template
    if email_address:
        request_template['email'] = email_address
    if key:
        request_template['key'] = key
    if param:
        request_template['param'] = param
    if begin_date:
        request_template['begin_date'] = begin_date
    if end_date:
        request_template['end_date'] = end_date
    if fips and len(fips)==5:
        request_template['state'] = fips[:2]
        request_template['county'] = fips[2:]            

    # Make sure there are values that allow us to make a call - these are always required
    if not request_template['email']:
        raise Exception("Must supply an email address to call 'request_daily_summary()'")
    if not request_template['key']: 
        raise Exception("Must supply a key to call 'request_daily_summary()'")
    if not request_template['param']: 
        raise Exception("Must supply param values to call 'request_daily_summary()'")
    if not request_template['begin_date']: 
        raise Exception("Must supply a begin_date to call 'request_daily_summary()'")
    if not request_template['end_date']: 
        raise Exception("Must supply an end_date to call 'request_daily_summary()'")
    # Note we're not validating FIPS fields because not all of the daily summary actions require the FIPS numbers
        
    # Compose the request
    request_url = endpoint_url+endpoint_action.format(**request_template)
        
    # Make the request
    try:
        # Wait first, to make sure we don't exceed a rate limit in case an exception occurs
        # During the request processing - throttling is always a good practice with a free data source
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

### Step 5: Generating the Final AQI Dataset

During initial analysis, it was found that the Gaseuous AQI data was missing for the city of Twin Falls, Idaho. It was also found that the AQI data for monitoring stations in Twin Falls county is available from 1986 onwards. Thus, even though our wildfires dataset ranges from the year 1963 to 2020, the AQI data is available from 1986 only.

Apart from that, since we are making a "daily" summary request, the AQI values need to be estimated on a yearly basis somehow. This can be done by taking an average of all the AQI data for that year or picking the maximum value as well. Since this analysis is concerned about the potential extreme impacts of wildfires on air quality and how high pollution events correlate with smoke estimates, considering the maximum AQI for each year felt more suitable. The document for the [Reporting of Daily Air Quality](https://www.airnow.gov/sites/default/files/2020-05/aqi-technical-assistance-document-sept2018.pdf) sheds some light on how AQI can be calculated across different pollutants. It suggests picking the maximum AQI value when multiple pollutants contribute towards the AQI. This thus further strengthens our approach of picking the maximum AQI value instead of average.

Finally, instead of collecting the entire year's data to estimate the annual AQI, the data is pulled only for the period of fire season. The annual fire season is said to run from May 1st through October 31st. Since our aim is to understand the smoke estimate's relation with AQI levels, picking the maximum AQI level from the fire season period every year seemed appropriate.

In [24]:
# Initialize a dataframe with year and AQI columns
aqi_df = pd.DataFrame(columns=['Year', 'AQI'])

# Initialize the year range and all the request parameters
year = list(np.arange(1986, 2021, 1))
request_data = AQS_REQUEST_TEMPLATE.copy()
request_data['email'] = USERNAME
request_data['key'] = APIKEY
request_data['state'] = CITY['fips'][:2]
request_data['county'] = CITY['fips'][2:]
request_data['param'] = AQI_PARAMS_PARTICULATES

# Iterate through every year
for y in year:
    
    # Initialize the start and end date based on the fire season
    begin_date = str(y)+'0501'
    end_date = str(y)+'1031'
    
    # Request daily summary data for the fire season for a specific year
    particulate_aqi = request_daily_summary(request_template=request_data, begin_date=begin_date, 
                                            end_date=end_date)
    # Set maximum AQI to zero
    max_aqi = 0
    
    # Iterate through the daily summary data from different monitoring stations
    for data in particulate_aqi["Data"]:
        
        # Check if the AQI value is greater than the current maximum AQI value
        if "aqi" in list(data.keys()):
            aqi = data["aqi"]
            if aqi is not None and aqi > max_aqi:
                max_aqi = aqi
    
    # Create a new row with the year and maximum AQI
    new_row = {'Year': y, 'AQI': max_aqi}
    # Append the new row to the DataFrame
    aqi_df =  pd.concat([aqi_df, pd.DataFrame([new_row])], ignore_index=True)

In [25]:
# Look at the final dataframe
aqi_df

,Year,AQI
0,1986,64
1,1987,75
2,1988,62
3,1989,64
4,1990,68
5,1991,72
6,1992,0
7,1993,0
8,1994,0
9,1995,81


It can be found that the AQI Data is missing for the years 1992, 1993, 1994 and 2014. These values are first replaced with an empty value instead of zero, and filled with the rolling average value of the five previous AQI values. While these values may not be accurate, considering five previous values' rolling average might be a reasonal estimate for the missing AQI data.

In [26]:
# Replace zeroes with empty values
aqi_df['AQI'] = aqi_df['AQI'].replace(0, None)

# Fill the empty values with the rolling average of the five previous AQI values
filled_column = aqi_df['AQI'].fillna(aqi_df['AQI'].rolling(5, min_periods=1).mean())

# Update the original column in the DataFrame
aqi_df['AQI'] = filled_column
aqi_df['AQI'] = aqi_df['AQI'].astype(int)

In [27]:
# Look at the updated dataframe
aqi_df

,Year,AQI
0,1986,64
1,1987,75
2,1988,62
3,1989,64
4,1990,68
5,1991,72
6,1992,66
7,1993,68
8,1994,70
9,1995,81


It can be observed that the missing AQI data has now been replaced with the rolling average values of the previous five AQI measurements. The dataset is now complete for the years 1986-2020, and can be exported to a CSV file.

In [28]:
# Save the final AQI dataframe to a CSV file
aqi_df.to_csv('Yearly_AQI_Data.csv', index=False)